In [2]:
import sys
sys.path.insert(0, '/home/jianx/search-exposure/forward_ranker/')
import torch
from train import generate_sparse
from load_data import obj_reader
import network
from annoy import AnnoyIndex

In [4]:
PASSAGE_BATCH_SIZE = 8

MODEL_PATH = "/home/jianx/data/results/100_1000_1000_0.001_256_10.model"
DEVICE = torch.device("cuda:1")
EMBED_SIZE = 256

In [9]:
NET = network.DSSM(embed_size=EMBED_SIZE)
NET.load_state_dict(torch.load(MODEL_PATH))
NET.to(DEVICE)
NET.eval()

DSSM(
  (model): Sequential(
    (0): Linear(in_features=100000, out_features=64, bias=True)
    (1): ReLU()
    (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=64, out_features=64, bias=True)
    (9): ReLU()
    (10): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (11): Dropout(p=0.1, inplace=False)
    (12): Linear(in_features=64, out_features=256, bias=True)
  )
)

In [ ]:
EMBEDDING_PATH = "/home/jianx/data/results/passage_embeddings.dict"
passage_embeddings = obj_reader(EMBEDDING_PATH)

In [ ]:
print(len(passage_embeddings[0]))

In [6]:
QUERY_TEST_DICT = obj_reader("/home/jianx/data/queries_test.dict")

In [ ]:
PASSAGE_DICT = obj_reader("/home/jianx/data/passages.dict")

In [ ]:
RATING_DICT = obj_reader("/home/jianx/data/rel_scores.dict")

### Brute Force Full Retrieval

In [ ]:
QIDS = list(RATING_DICT.keys())
QUERY_EMBEDDINGS = []
for qid in QIDS:
    q_seq = QUERY_TEST_DICT[qid]
    QUERY_EMBEDDINGS.append(NET(generate_sparse(q_seq).to(DEVICE)).detach())
QUERY_EMBEDDING_TENSOR = torch.stack(QUERY_EMBEDDINGS, dim=0).unsqueeze(dim=1)

In [ ]:
PASSAGE_EMBEDDING_BATCHES = []
for i in range(PASSAGE_BATCH_SIZE):


In [ ]:
# compute cosine similarity
result_dict = {}
for i, qid in enumerate(QIDS):
    q_embed = NET(QUERIES[i]).detach()
    q_results = {}
    for item in passage_embeddings.items():
        p_embed = torch.tensor(item[1]).to(DEVICE)
        score = torch.cosine_similarity(q_embed.unsqueeze(0), p_embed.unsqueeze(0)).item()
        q_results[item[0]] = score
    result_dict[qid] = q_results

In [1]:
print(result_dict[19335])

NameError: name 'result_dict' is not defined

### Annoy Full Retrieval

In [7]:
PASSAGE_INDEX = AnnoyIndex(EMBED_SIZE, 'euclidean')
PASSAGE_INDEX.load("/home/jianx/data/annoy/100_passage_index.ann")
PID_MAPPING = obj_reader("/home/jianx/data/annoy/100_pid_map.dict")

In [18]:
qids = []
queries = []
for key, value in QUERY_TEST_DICT.items():
    qid = key
    q_seq = value
    if q_seq != [] and qid not in qids:
        qids.append(qid)
        queries.append(generate_sparse(q_seq).to(DEVICE))
# compute cosine similarity
result_dict = {}
for i, qid in enumerate(qids):
    top_list = PASSAGE_INDEX.get_nns_by_vector(NET(generate_sparse(QUERY_TEST_DICT[qid]).to(DEVICE)).detach(),1000)
    print(top_list)
    for j, pid in enumerate(top_list):
        if pid in PID_MAPPING:
            top_list[j] = PID_MAPPING[pid]
        else:
            print("Not captured in mapping")
            top_list[j] = -1
    q_embed = NET(queries[i]).detach()
    q_results = {}
    score = float(len(top_list))
    for j, pid in enumerate(top_list):
        q_results[pid] = score
        score -= 1
    result_dict[qid] = q_results

[2473838, 7535182, 2757198, 1249083, 1888520, 5327992, 7060050, 790712, 885221, 1661368, 1808369, 2142949, 6157273, 6655820, 130461, 986426, 4731791, 5968117, 1336028, 8736038, 2627327, 6278519, 2510957, 416375, 1812914, 7862491, 788049, 5987267, 5276991, 925803, 7502217, 3633110, 1260273, 5914710, 2174497, 8480568, 6527022, 2448185, 1342881, 7796812, 7994550, 1277628, 2855997, 8813087, 1005792, 4034572, 2707851, 4408310, 4202570, 4779155, 2007808, 344836, 526517, 7707268, 2558475, 5245384, 6621583, 2110865, 5155049, 1141258, 5611816, 2082481, 4607778, 7740073, 6775985, 4949599, 2534183, 3189399, 4136068, 3583202, 5179352, 182896, 3238010, 1137966, 3795872, 5016311, 7694456, 160001, 7008026, 2398322, 5403943, 3880469, 2211499, 6729371, 752019, 4288935, 2692417, 6398377, 1829168, 7600976, 7552025, 7413913, 6715459, 1388897, 4234581, 7667207, 6140217, 2005733, 4202574, 5166781, 6436197, 4520837, 3043705, 7818378, 2334370, 6053738, 6197854, 7867753, 900077, 1403004, 3470611, 7527768, 2334

In [14]:
brute_result = obj_reader("/home/jianx/data/results/brute_search_result.dict")

In [22]:
precision = 0
for qid, top_1000_list in brute_result.items():
    print(len(top_1000_list))
    annoy_top_1000 = result_dict[qid]
    precision += 1- len(set(top_1000_list) - set(annoy_top_1000))/len(top_1000_list)
precision /= len(brute_result)
print(precision)

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
0.011441860465116289


In [16]:
import random
print(sorted(result_dict[qid].items(), key=lambda x: (x[1], [-1, 1][random.randrange(2)]), reverse=True))

[(48692, 0.8087702393531799), (53746, 0.7796605825424194), (49883, 0.7765213251113892), (28590, 0.7714456915855408), (37478, 0.7576753497123718), (52548, 0.7563543915748596), (65233, 0.7527650594711304), (31695, 0.7502312064170837), (12036, 0.7473176121711731), (7941942, 0.6197816729545593), (7504098, 0.5882886648178101), (4464782, 0.5501894354820251), (8100035, 0.5469191670417786), (7742740, 0.5462336540222168), (3201976, 0.5367746949195862), (2387062, 0.5219270586967468), (5271926, 0.521277904510498), (3151942, 0.5130279064178467), (1253640, 0.504011332988739), (3152802, 0.4938577115535736), (847991, 0.4745821952819824), (2949501, 0.4648352265357971), (3466047, 0.46283942461013794), (3601105, 0.4607013761997223), (7161832, 0.44946643710136414), (3869661, 0.4384290277957916), (1326788, 0.4344586133956909), (4696015, 0.43377187848091125), (8603392, 0.4314953684806824), (2287080, 0.42788517475128174), (8247225, 0.4277639091014862), (5689315, 0.4156720042228699), (1573378, 0.413655847311

In [22]:
from test import get_ndcg_precision_rr
import numpy as np
rating_dict = obj_reader("/home/jianx/data/rel_scores.dict")
rank = 10

qids = list(result_dict.keys())
result_ndcg = []
result_prec = []
result_rr = []
for qid in qids:
    if qid in rating_dict:
        ndcg, prec, rr = get_ndcg_precision_rr(rating_dict[qid], result_dict[qid], rank)
        result_ndcg.append(ndcg)
        result_prec.append(prec)
        result_rr.append(rr)
avg_ndcg = np.nanmean(result_ndcg)
avg_prec = np.nanmean(result_prec)
avg_rr = np.nanmean(result_rr)
print(avg_ndcg, avg_prec, avg_rr)

0.010260382935891153 0.009302325581395349 0.6888888888888888


In [6]:
print(len(set(w.lower() for w in open('/home/jianx/data/processed_passages.txt').read().split())))

100000
